In [ ]:
from baynes import *
import pandas as pd
from cmdstanpy import CmdStanModel
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import cmdstanpy
from baynes import hdi
from collections import defaultdict
import json
from collections import defaultdict
import arviz as az
import numpy as np

cmdstanpy.utils.get_logger().setLevel(logging.ERROR)

sns.set_style('ticks')
sns.set_context("notebook", font_scale=1.2)
plt.rc("axes.spines", top=False, right=False)


In [ ]:
fit=cmdstanpy.from_csv("/home/pietro/work/TESI/baynes/data/endpoint/64det/")

In [ ]:
ax = sns.catplot(fit.draws_pd(['E_syst']), kind='box')
ax.figure.set_size_inches(20,5)

In [ ]:
import json


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def save_dict_to_json(dd, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(dd, json_file, indent=4, cls=NumpyEncoder)
    print(f"Dictionary saved to {file_path}")

file_path = "/home/pietro/work/TESI/baynes/data/endpoint/64results.json"



In [ ]:
m = 0
Q = 2833
A_Ho = 1
bkg = 1e-4
pu = 1.5e-6
FWHM=6.8
n_det = 64
n_days = 100

Qs=[]
FWHMs=[]
N_evs=[]
events = []
syst=[]
E_H = [2047, 1842, 414.2, 333.5, 49.9, 26.3]

data=defaultdict(list)
fig, ax = plt.subplots()
for i in range(n_det):
    E_shift = np.random.normal(0, 10)
    FWHM=np.random.normal(6.8, 1)

    s = SpectraSampler({'$^{163}Ho$': [probability.HoSpectrum, [m, Q+E_shift, [E+E_shift for E in E_H]], np.random.normal(1,0.2)]}, flat_bkg=bkg, f_pileup=pu, FWHM=FWHM, dE=2, integrate=False, ROI=[2600, 2950])
    s.set_measure_time(n_days, n_det=1)

    events.append(s.sample()[0])
    N_evs.append(sum(events[-1]))
    data['FWHM'].append(FWHM)
    syst.append(E_shift)
    s.plot_events(events[-1], ax=ax)
ax.get_legend().remove()

FWHMs=  [x for _, x in sorted(zip(N_evs, data['FWHM']), key=lambda pair: pair[0], reverse=True)]
events = [x for _, x in sorted(zip(N_evs, events), key=lambda pair: pair[0], reverse=True)]
N_evs = sorted(N_evs, reverse=True)
data.update({'N_bins': len(events[1]), 
      'N_det': n_det,
      'x': s.ROI_bin_edges, 
      'counts': events,
      'N_ev': N_evs,
      'counts1': events[0],
      'p_Q': 2833,
      'm_max':250,

})
save_dict_to_json(data, "/home/pietro/work/TESI/baynes/data/endpoint/data.json")



In [ ]:
def combine_detectors(events, N_ev, FWHMs, n_groups):
    L = len(N_evs)
    n_in_group = int(L/n_groups)
    evs = [sum(events[i*n_in_group:(i+1)*n_in_group]) for i in range(n_groups)]
    Ns = [sum(N_ev[i*n_in_group:(i+1)*n_in_group]) for i in range(n_groups)]
    FWs = [sum(FWHMs[i*n_in_group:(i+1)*n_in_group])/n_in_group for i in range(n_groups)]
    return evs, Ns, FWs

In [ ]:
n_groups=1
cgroup, Ngroup, FWgroup = combine_detectors(events, N_evs, FWHMs, n_groups)
print(Ngroup)

data.update({'N_det': int(n_det/n_groups), 'counts':cgroup, 'N_ev': Ngroup, 'counts1':cgroup[0], 'FWHM':FWgroup})

In [ ]:
Ngroup

In [ ]:
stan_file= get_stan_file("Ho_endpoint_simple.stan")
model = CmdStanModel(stan_file=stan_file,
                     **get_compiler_kwargs())

n_groups=1
cgroup, Ngroup, FWgroup = combine_detectors(events, N_evs, FWHMs, n_groups)
print(Ngroup)

data.update({'N_det': 1, 'counts':cgroup[0], 'N_ev': Ngroup[0],  'p_FWHM':FWgroup[0]})

sampler_kwargs={
    'chains': 4,
   # 'iter_warmup': 600,
    'iter_sampling': 3000,
    'save_warmup': True, 
   # 'max_treedepth': 11
    'inits': {'m_red': 0.001, 'z': 1, 'bkg': 0.004, 'FWHM': 6},
    'adapt_delta':0.95,
    #'show_console': True
}

plot_pars = ['m_nu', 'Q', 'f_bkg', 'FWHM']
p = FitPlotter(col_wrap=4, output_dir="/home/pietro/work/TESI/thesis/figures/ch3/endpoint/", output_format='.pdf')
fit = standard_analysis(model, data, p, sampler_kwargs=sampler_kwargs, plot_params=plot_pars)
fit.save_csvfiles("/home/pietro/work/TESI/baynes/data/endpoint/"+str(n_groups)+"/")

In [ ]:
stan_file= get_stan_file("Ho_endpoint_multidet.stan")
multi_model = CmdStanModel(stan_file=stan_file,
                     **get_compiler_kwargs())

for n_groups in [2,4,8, 16, 32, 64]:
    cgroup, Ngroup, FWgroup = combine_detectors(events, N_evs, FWHMs, n_groups)
    print(Ngroup)

    data.update({'N_det': n_groups, 'counts':cgroup, 'N_ev': Ngroup,  'FWHM':FWgroup, 'counts1':cgroup[0]})





    sampler_kwargs={
        'chains': 24,
        'iter_warmup': 600,
        'iter_sampling': 500,
        'save_warmup': True, 
        'adapt_delta': 0.95,
        'inits': {'m_red': 0.001, 'z': 0, 'f_bkg': [0.004]*n_groups, 'E_sigma':1, 'E_syst':[0]*n_groups},

    }

    pars=['m_nu', 'Q', 'E_sigma', 'E_syst[1]', 'f_bkg[1]']
    fit = standard_analysis(multi_model, data, p, sampler_kwargs, fit_title='m=0', data_key='counts1', rep_key='counts1_rep', plot_params=pars)
    fit.save_csvfiles("/home/pietro/work/TESI/baynes/data/endpoint/"+str(n_groups)+"/")

In [ ]:
df = pd.DataFrame()
for i in [1,2,4,8,16,32, 64]:
    if i ==64:
        fit=cmdstanpy.from_csv('/home/pietro/work/TESI/baynes/data/endpoint/64det')
    else:
        fit = cmdstanpy.from_csv('/home/pietro/work/TESI/baynes/data/endpoint/'+str(i)+'/')
    draws=fit.draws_pd('m_nu')
    draws['group']=i
    df =pd.concat([df, draws])